## Queries - LSSTCam

In this notebook, we show how to query the LSSTCam repository\
and view the resulting images.\

Craig Lage - 09-Sep-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.summit.utils.plotting import plot
import lsst.afw.cameraGeom.utils as camGeomUtils
import lsst.afw.math as afwMath
import lsst.afw.display as afwDisplay

In [ ]:
butler = Butler('embargo_new', collections=['LSSTCam/raw/all', 'LSSTCam/calib/unbounded'])

## First, get a list of exposures
### These should match what you see in RubinTV.

In [ ]:
dayObs = 20240906
instrument = "LSSTCam"

exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
                    where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter)


# Get the data from the headers

In [ ]:
expId = 2024090600117
detector = 111
instrument = 'LSSTCam'
mData = butler.get('raw.metadata', detector=detector, exposure=expId, instrument=instrument)
for key in mData.keys():
    print(key, mData[key])

# Now get the raw data

## Next, look at the raw data from one of the exposures.
### Because of the large pedestal, we don't see much.  We need to do ISR

In [ ]:
expId = 2024090600117
detector = 111
instrument = 'LSSTCam'
raw = butler.get('raw', detector=detector, exposure=expId, instrument=instrument)


In [ ]:
%matplotlib inline        
x = plot(raw, stretch='ccs')
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/LSSTCam_raw_{detector}_{expId}.png")

## Define a simple ISR
### Just overscan subtraction and bias subtraction.

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.usePtcGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

## Run the ISR and look at the result

In [ ]:
expId = 2024090600117
detector = 111
instrument = 'LSSTCam'
exp = butler.get('raw', detector=detector, exposure=expId, instrument=instrument)
isrResult = isrTask.run(exp) # This runs the ISR

In [ ]:
%matplotlib inline
x = plot(isrResult.exposure, stretch='linear')
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/LSSTCam_{detector}_{expId}.png")

# Plot a small region

In [ ]:
x = plot(isrResult.exposure.image.array[2200:2300, 2600:2700])

# Get the value of a particular pixel

In [ ]:
exp.image.array[2100, 2100]

# Plot the cross-chip uniformity

In [ ]:
plt.title(f"LSSTCam {expId}, Bias")
plt.plot(isrResult.exposure.image.array[2100, :], label='X cut')
plt.plot(isrResult.exposure.image.array[:, 2100], label='Y cut')
plt.ylim(-20, 20)
plt.ylabel("Flux (electrons)")
plt.xlabel("Pixels")
plt.legend()
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/LSSTCam_Slices_{detector}_{expId}.png")

# Now assemble all CCDs and plot the result

In [ ]:
def myCallback(im, ccd, imageSource):
    """Assemble the CCD image and do basic ISR"""
    dayObs = imageSource.kwargs['day_obs']
    seqNum = imageSource.kwargs['seq_num']
    exp = imageSource.butler.get('raw', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    isrResult = isrTask.run(exp)
    nQuarter = ccd.getOrientation().getNQuarter()
    oim = afwMath.rotateImageBy90(isrResult.exposure.image, nQuarter)
    return oim

def rawCallback(im, ccd, imageSource):
    """Assemble the CCD image.  Just bias subtraction and gain correction"""
    oim = camGeomUtils.rawCallback(im, ccd, imageSource,
                                   subtractBias=False, correctGain=False)
    return oim

In [ ]:
%matplotlib inline
instrument = "LSSTCam"
camera = butler.get('camera', instrument=instrument)
fig = plt.figure(figsize=(12,12))
import lsst.afw.display as afwDisplay
disp = afwDisplay.Display(1, "matplotlib")
disp.scale('linear', min=-0.2, max=0.2)
dayObs = 20240906
seqNum = 117
dataType='raw'
mos = camGeomUtils.showCamera(camera,
                              camGeomUtils.ButlerImage(butler, dataType, 
                                                       instrument=instrument, 
                                                       day_obs=dayObs, seq_num=seqNum,
                                                       verbose=False, callback=myCallback,
                                                       background=np.nan),
                              binSize=16, display=disp, overlay=False,
                              title="%d %d" % (dayObs, seqNum))

plt.savefig(f"/home/c/cslage/u/LSSTCam/images/LSSTCam_Test_16_{dayObs}_{seqNum}.png")

# Plot the cross-detector uniformity

In [ ]:
%matplotlib inline
plt.title(f"LSSTCam {dayObs}_{seqNum}")
plt.plot(mos.array[2000, :], label="X slice")
plt.plot(mos.array[:, 2000], label="Y slice")
plt.ylim(-5, 5)
plt.ylabel("Flux (ADU)")
plt.xlabel("Pixels (binned by 16)")
plt.legend()
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/LSST_Slices_{dayObs}_{seqNum}.png")